# **8-applied-statistics. Lab 2**

Михайло Корешков  
Валентина Сириця  
ФІ-91  


In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math

In [2]:
# L for Lambda
def generate(n, L):
    z = np.random.rand(n)
    return -np.log(z)/L

# create a function
def create_F_exp(L):
    def F(u):
        return 1 - np.exp(-L*u)
    return F


Ns = [ 1000, 10_000, 100_000 ]


## Завдання 1. Критерій Колмогорова

a)

In [3]:
# F should be numpy-compatible
def kolmohorov_test(F, X, gamma=0.05):
    def stat(F, sorted_X):
        n = sorted_X.shape[0]
        k = np.arange(1, n+1) / n
        invn = 1/n
        D = np.max([ F(sorted_X)-k-invn , k-F(sorted_X) ])
        return D
    
    # 'Limiting distribution of scaled Kolmogorov-Smirnov two-sided test statistic'
    z = sp.stats.kstwobign().ppf(1-gamma)  

    oX = X.sort()  # ordered X
    D = stat(F, X)
    val = np.sqrt(X.shape[0]) * D
    return (val < z, val, z)

In [9]:
print("1.a)")
print("H0: X ~ Exp(1);  Truth is X ~ Exp(1)")
print("Using Kolmohorov test")
for n in Ns:
    X = generate(n, 1)
    result, val, crit_val = kolmohorov_test(create_F_exp(1), X)
    print(f"> N={n}; result={result} (val={val}, crit_val={crit_val})")

1.a)
H0: X ~ Exp(1);  Truth is X ~ Exp(1)
Using Kolmohorov test
> N=1000; result=True (val=0.6586907024383176, crit_val=1.3580986393225505)
> N=10000; result=True (val=0.8697833752683893, crit_val=1.3580986393225505)
> N=100000; result=True (val=1.2441612071627677, crit_val=1.3580986393225505)


In [10]:
print("1.b)")
print("H0: X ~ Exp(1);  Truth is X ~ Exp(1.3)")
print("Using Kolmohorov test")
for n in Ns:
    X = generate(n, 1.3)
    result, val, crit_val = kolmohorov_test(create_F_exp(1), X)
    print(f"> N={n}; result={result} (val={val}, crit_val={crit_val})")

1.b)
H0: X ~ Exp(1);  Truth is X ~ Exp(1.3)
Using Kolmohorov test
> N=1000; result=False (val=2.3460517408631056, crit_val=1.3580986393225505)
> N=10000; result=False (val=10.934916917999804, crit_val=1.3580986393225505)
> N=100000; result=False (val=30.79624416861786, crit_val=1.3580986393225505)


In [4]:
X = generate(150, 1.3)
result, val, crit_val = kolmohorov_test(create_F_exp(1), X)
print(f"> N={150}; result={result} (val={val}, crit_val={crit_val})")

> N=150; result=False (val=1.383286332330551, crit_val=1.3580986393225505)


## Завдання 2. Критерій Хі-квадрат

In [6]:
# F should be numpy-compatible
def chi2_test(F, X, r=None, gamma=0.05):
    n = X.shape[0]
    r = r if r is not None else int(3*n/100)
    z = sp.stats.chi2.ppf(1-gamma, r-1)

    Y = F(X)  # uniform on [0;1] 
    v = np.zeros(r)
    for y in Y: 
        v[int(y*r)] += 1
    
    d = (np.square(v)*r/n).sum() - n
    return (d < z, d, z)

In [12]:
print("2.a)")
print("H0: X ~ Exp(1);  Truth is X ~ Exp(1)")
print("Using Chi2 test")
for n in Ns:
    X = generate(n, 1)
    r = int(30*n/1000)
    result, val, crit_val = chi2_test(create_F_exp(1), X, r)
    print(f"> N={n}; result={result} (val={val}, crit_val={crit_val})")

2.a)
H0: X ~ Exp(1);  Truth is X ~ Exp(1)
Using Chi2 test
> N=1000; result=True (val=32.600000000000136, crit_val=42.55696780429269)
> N=10000; result=True (val=326.1800000000003, crit_val=340.3278503488595)
> N=100000; result=True (val=2967.140000000014, crit_val=3127.515431571307)


In [13]:
print("2.b)")
print("H0: X ~ Exp(1);  Truth is X ~ Exp(1.3)")
print("Using Chi2 test")
for n in Ns:
    X = generate(n, 1.3)
    r = int(30*n/1000)
    result, val, crit_val = chi2_test(create_F_exp(1), X, r)
    print(f"> N={n}; result={result} (val={val}, crit_val={crit_val})")

2.b)
H0: X ~ Exp(1);  Truth is X ~ Exp(1.3)
Using Chi2 test
> N=1000; result=False (val=78.5, crit_val=42.55696780429269)
> N=10000; result=False (val=939.8600000000006, crit_val=340.3278503488595)
> N=100000; result=False (val=8784.259999999995, crit_val=3127.515431571307)


In [7]:
n = 200
X = generate(n, 1.3)
r = int(30*n/1000)
result, val, crit_val = chi2_test(create_F_exp(1), X, r)
print(f"> N={n}; result={result} (val={val}, crit_val={crit_val})")

> N=200; result=False (val=30.880000000000024, crit_val=11.070497693516351)


## Завдання 3. Критерій пустих ящиків

In [14]:
# F should be numpy-compatible 
def emptybox_test(F, X, r=None, gamma=0.05):
    n = X.shape[0]
    r = r if r is not None else int(n/2)
    
    # n/r -> rho. That is, rho = 2
    rho = 2
    z = sp.stats.norm.ppf(1 - gamma)
    k = r * np.exp(-rho) + z * np.sqrt(  r*np.exp(-rho) * ( 1 - (1+rho)*np.exp(-rho) )  )

    Y = F(X) # uniform on [0;1]
    v = np.zeros(r)
    for y in Y: 
        v[int(y*r)] += 1
    
    n_empty = (v == 0).sum()

    return (n_empty < k, n_empty, k)

In [15]:
print("3.a)")
print("H0: X ~ Exp(1);  Truth is X ~ Exp(1)")
print("Using Empty Boxes test")
for n in Ns:
    X = generate(n, 1)
    r = int(n/2)
    result, val, crit_val = emptybox_test(create_F_exp(1), X, r)
    print(f"> N={n}; result={result} (val={val}, crit_val={crit_val})")

3.a)
H0: X ~ Exp(1);  Truth is X ~ Exp(1)
Using Empty Boxes test
> N=1000; result=True (val=65, crit_val=78.09582983486621)
> N=10000; result=True (val=689, crit_val=709.653242816322)
> N=100000; result=True (val=6763, crit_val=6871.0460439962335)


In [16]:
print("3.b)")
print("H0: X ~ Exp(1);  Truth is X ~ Exp(1.3)")
print("Using Empty Boxes test")
for n in Ns:
    X = generate(n, 1.3)
    r = int(n/2)
    result, val, crit_val = emptybox_test(create_F_exp(1), X, r)
    print(f"> N={n}; result={result} (val={val}, crit_val={crit_val})")

3.b)
H0: X ~ Exp(1);  Truth is X ~ Exp(1.3)
Using Empty Boxes test
> N=1000; result=True (val=78, crit_val=78.09582983486621)
> N=10000; result=False (val=784, crit_val=709.653242816322)
> N=100000; result=False (val=7807, crit_val=6871.0460439962335)


### Демонстрація складності прямого обчислення критичних значень для критерію пустих ящиків

In [17]:
def get_emptybox_crit_val(n,r,gamma=0.05):
    def calc_q(k, n, r):
        res = 0
        one = 1
        comb = math.comb(r,k)
        for j in range(0, r-k+1):
            res += comb * math.comb(r-k, j) * np.float_power(1 - (k+j)/r, n) * one
            one *= -1
        return res
    
    print(f'start for get_emptybox_crit_val(n={n}, r={r}, gamma={gamma})')

    q = np.zeros(r+1)
    for k in range(0, r+1):
        q[k] = calc_q(k, n, r)
    
    if r < 100:
        print("q:")
        print(q)
    
    print('> phase 1')

    q = np.flip(np.flip(q).cumsum())

    if r < 100:
        print("q.cumsum:")
        print(q)

    print('> phase 2')
    min_k = r
    for k in range(0, r+1):
        if q[k] < gamma and k < min_k:
            min_k = k
    
    print(f'> answer: k={min_k} for sum={q[min_k]}')
    return min_k

In [18]:
emptybox_crit_vals = dict()
for n in Ns:
    r = int(n / 2)
    emptybox_crit_vals[n] = get_emptybox_crit_val(n, r)


start for get_emptybox_crit_val(n=1000, r=500, gamma=0.05)
> phase 1
> phase 2
> answer: k=0 for sum=-3.0656435897757865e+26
start for get_emptybox_crit_val(n=10000, r=5000, gamma=0.05)


OverflowError: int too large to convert to float

## Завдання 4. Критерій однорідності Смирнова

In [ ]:
def smyrnov_test(X, Y, gamma):
    n, m = X.shape[0], Y.shape[0]
    X.sort()
    Y.sort()
    
    # 'Limiting distribution of scaled Kolmogorov-Smirnov two-sided test statistic'
    z = sp.stats.kstwobign().ppf(1-gamma)
    z = z * np.sqrt(1/n + 1/m)

    # ecdfx = empirical cumulative distribution function for X
    def ecdfx(u): return np.searchsorted(X, u, 'right') / n

    ecdfx_of_y = ecdfx(Y)
    k = np.arange(1, m+1)
    D_pos = np.max(k/m - ecdfx_of_y)
    D_neg = np.max(ecdfx_of_y - (k-1)/m)
    D = max(D_pos, D_neg)

    return (D < z, D, z)

In [ ]:
print("4.a)")
print("H0: X ~ Exp(1) and Y ~ Exp(1);  Truth is X, Y ~ Exp(1)")
print("Using Smyrnov test")
for n in Ns:
    X = generate(n, 1)
    Y = generate(int(n/2), 1)
    result, val, crit_val = smyrnov_test(X, Y, 0.05)
    print(f"> (X,Y): N={n}; result={result} (val={val}, crit_val={crit_val})")
    result, val, crit_val = smyrnov_test(Y, X, 0.05)
    print(f"> (Y,X): N={n}; result={result} (val={val}, crit_val={crit_val})")

4.a)
H0: X ~ Exp(1) and Y ~ Exp(1);  Truth is X, Y ~ Exp(1)
Using Smyrnov test
> (X,Y): N=1000; result=True (val=0.05399999999999999, crit_val=0.07438612600740335)
> (Y,X): N=1000; result=True (val=0.05399999999999999, crit_val=0.07438612600740335)
> (X,Y): N=10000; result=True (val=0.010400000000000076, crit_val=0.02352295844996817)
> (Y,X): N=10000; result=True (val=0.010400000000000076, crit_val=0.02352295844996817)
> (X,Y): N=100000; result=True (val=0.0046700000000000075, crit_val=0.007438612600740335)
> (Y,X): N=100000; result=True (val=0.0046700000000000075, crit_val=0.007438612600740335)


In [ ]:
print("4.b)")
print("H0: X ~ Exp(1) and Y ~ Exp(1);  Truth is X ~ Exp(1), but Y ~ Exp(1.3)")
print("Using Smyrnov test")
for n in Ns:
    X = generate(n, 1)
    Y = generate(int(n/2), 1.3)
    result, val, crit_val = smyrnov_test(X, Y, 0.05)
    print(f"> (X,Y): N={n}; result={result} (val={val}, crit_val={crit_val})")
    result, val, crit_val = smyrnov_test(Y, X, 0.05)
    print(f"> (Y,X): N={n}; result={result} (val={val}, crit_val={crit_val})")

4.b)
H0: X ~ Exp(1) and Y ~ Exp(1);  Truth is X ~ Exp(1), but Y ~ Exp(1.3)
Using Smyrnov test
> (X,Y): N=1000; result=False (val=0.13200000000000006, crit_val=0.07438612600740335)
> (Y,X): N=1000; result=False (val=0.13200000000000006, crit_val=0.07438612600740335)
> (X,Y): N=10000; result=False (val=0.08930000000000005, crit_val=0.02352295844996817)
> (Y,X): N=10000; result=False (val=0.08930000000000005, crit_val=0.02352295844996817)
> (X,Y): N=100000; result=False (val=0.09494000000000002, crit_val=0.007438612600740335)
> (Y,X): N=100000; result=False (val=0.09494000000000002, crit_val=0.007438612600740335)
